In [ ]:
#1º Etapa
import pandas as pd 
treino = pd.read_csv('treino.csv') # Lendo os arquivos
teste = pd.read_csv('teste.csv') # Lendo os arquivos

In [ ]:
#Alterando os nomes das colunas “temratura-50m” e “velocidde-vento-25m” para “temperatura-50m” e “velocidade-vento-25m”;
treino.rename(columns={'temratura-50m':'temperatura-50m','velocidde-vento-25m': 'velocidade-vento-25m'}, inplace=True)
teste.rename(columns={'temratura-50m':'temperatura-50m','velocidde-vento-25m': 'velocidade-vento-25m'}, inplace=True)

In [ ]:
#Exiba os 5 primeiros elementos do dataset de treino e de teste;
treino.head() 
teste.head()

In [ ]:
#Exiba a quantidade total de elementos nas tabelas 
#e a média dos valores da coluna velocidade-vento-50m;
treino.count().sum() # retorna o total de elementos da tabela
teste.count().sum() # retorna o total de elementos da tabela
treino['velocidade-vento-50m'].mean()
teste['velocidade-vento-50m'].mean()

In [ ]:
#Exiba qual Data apresentou o maior valor de velocidade-vento-50m;
treino['velocidade-vento-50m'].max()
teste['velocidade-vento-50m'].max()

In [ ]:
#gráficos interessante para o problema em questão e justifique a escolha deles
import matplotlib.pyplot as plt
plt.figure(figsize = (15,10))
plt.plot(treino[["velocidade-vento-50m"]], label='Velocidade-vento-50m')
plt.legend(loc='best')
plt.show()

import seaborn as sns

sns.pairplot(treino[["velocidade-vento-50m"]], diag_kind="kde")
train_stats = treino[["velocidade-vento-50m"]].describe()
train_stats.pop("velocidade-vento-50m")
train_stats = train_stats.transpose()
train_stats

In [ ]:
#2º Etapa
#Tratamento de dados para lidar com os dados faltantes;
valores_ausentes = treino.isnull()
medias = treino.mean() #média da coluna
treino.fillna(medias, inplace=True)
treino.isna().sum()

valores_ausentes = teste.isnull()
medias = teste.mean() #média da coluna
teste.fillna(medias, inplace=True)
teste.isna().sum() #Verificando se existe valores nulos ou contém alguns valores não conhecidos (unknown).

In [ ]:
treino['data-hora'] = pd.to_datetime(treino['data-hora'])
treino.set_index('data-hora', inplace=True)

teste['data-hora'] = pd.to_datetime(teste['data-hora'])
teste.set_index('data-hora', inplace=True)

vel = ["velocidade-vento-50m"]
treino = treino.loc[:, vel]
treino.head()


In [ ]:
#Escalone os dados de entrada para que fiquem dentro da faixa de valores entre 0 e 1;

from sklearn.preprocessing import MinMaxScaler
scl = MinMaxScaler(feature_range= (0,1))
treino = scl.fit_transform(treino)
teste = scl.fit_transform(teste)
arrayTr = treino
arrayTe = teste

In [ ]:
#Divida os dados de treinamento em X_train e y_train
look_back = 20 #dias anteriores
forward_hours = 3 #prever o que acontecerá com o valor n horas(3 próximas horas)
num_periods = 10 # períodos (num_periods) para testar o modelo

#A cada período, o modelo irá prever os próximos n dias.
#O resto será utilizado para o treinamento (Treino e Validação).

div = len(arrayTr) - num_periods*forward_hours

array_test = arrayTe[:]
array_train = arrayTr[:]


In [ ]:
# salvar modelo
import pickle
# salvar o array no arquivo norm.pkl
pickle.dump(scl, open('Escalonador.pkl', 'wb'))

In [ ]:
#Dados na entrada X e na saída Y, dividindo em 'n' dias anteriores como entrada X 
#e 'm' próximos dias como Y
import numpy as np
def processData(data, look_back, forward_hours,jump=1):
    X,Y = [],[]
    for i in range(0,len(data) -look_back -forward_hours +1, jump):
        X.append(data[i:(i+look_back)])
        Y.append(data[(i+look_back):(i+look_back+forward_hours)])
    return np.array(X),np.array(Y)


#Divida os dados de teste em X_test e y_test


X_test,y_test = processData(array_test,look_back,forward_hours,forward_hours)
y_test = np.array([list(a.ravel()) for a in y_test])

X,y = processData(array_train,look_back,forward_hours)
y = np.array([list(a.ravel()) for a in y])

#Separe uma parte dos dados de X_train e y_train para validação (X_val e y_val);

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)


In [ ]:
#3º Etapa
#Desenvolva um modelo que aprenda a predizer a velocidade-vento-50m
# das 3 próximas horas a partir dos dados de treinamento
from keras.models import Sequential
from keras.layers import LSTM,Dense
import matplotlib.pyplot as plt


# In[63]:


NUM_NEURONS_FirstLayer = 25
NUM_NEURONS_SecondLayer = 15
EPOCHS = 25

#Build the model
model = Sequential()
model.add(LSTM(NUM_NEURONS_FirstLayer,input_shape=(look_back,1), return_sequences=True))
model.add(LSTM(NUM_NEURONS_SecondLayer,input_shape=(NUM_NEURONS_FirstLayer,1)))
model.add(Dense(forward_hours))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train,y_train,epochs=EPOCHS,validation_data=(X_validate,y_validate),shuffle=True,batch_size=2, verbose=2)

In [ ]:
#Apresente o gráfico de loss de treinamento e validação do modelo. 
#Mostre que o modelo não apresenta over/underfitting;

plt.figure(figsize = (15,10))

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend(loc='best')
plt.show()


In [ ]:
#Apresente gráficos e tabelas relacionados ao desempenho do modelo desenvolvido, explicando os resultados

Xt = model.predict(X_test)
#prever os dados de Teste para o resultado
plt.figure(figsize=(15, 10))

for i in range(0, len(Xt)):
    plt.plot([x + i * forward_hours for x in range(len(Xt[i]))], scl.inverse_transform(Xt[i].reshape(-1, 1)), color='r')

plt.plot(0, scl.inverse_transform(Xt[i].reshape(-1, 1))[0], color='r', label='Prediction')  # only to place the label

plt.plot(scl.inverse_transform(y_test.reshape(-1, 1)), label='Target')
plt.legend(loc='best')
plt.show()

division = len(array) - num_periods*forward_hours

leftover = division%forward_hours+1

array_test = array[division-look_back:]
array_train = array[leftover:division]

Xtrain,ytrain = processData(array_train,look_back,forward_hours,forward_hours)
Xtest,ytest = processData(array_test,look_back,forward_hours,forward_hours)


Xtrain = model.predict(Xtrain)
Xtrain = Xtrain.ravel()

Xtest = model.predict(Xtest)
Xtest = Xtest.ravel()

y = np.concatenate((ytrain, ytest), axis=0)

plt.figure(figsize = (15,10))

# Data in Train/Validation
plt.plot([x for x in range(look_back+leftover, len(Xtrain)+look_back+leftover)], scl.inverse_transform(Xtrain.reshape(-1,1)), color='r', label='Train')
# Data in Test
plt.plot([x for x in range(look_back +leftover+ len(Xtrain), len(Xtrain)+len(Xtest)+look_back+leftover)], scl.inverse_transform(Xtest.reshape(-1,1)), color='y', label='Test')

#Data used
#plt.plot([x for x in range(look_back+leftover, look_back+leftover+len(Xtrain)+len(Xtest))], scl.inverse_transform(y.reshape(-1,1)), color='b', label='Target')

#Initial data. It should overlap the data used
plt.plot(scl.inverse_transform(array), color='g', label='Esperado')

plt.legend(loc='best')
plt.show()


In [ ]:
#Avalie quantitativamente o desempenho final do modelo desenvolvido a partir dos dados de teste, 
# utilizando as métricas Mean Squared Error (MSE), Mean Absolute Error (MAE), coeficiente de correlação Pearson r e o coeficiente de determinação R²;

